Читання та відображення вихідного файлу;

In [1]:
import cv2

img = cv2.imread('./road2.jpg')

cv2.imshow('Input image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Конвертація в сіре зображення;

In [3]:
import cv2

img = cv2.imread('./road2.jpg')

grayScale = cv2.cvtColor(
    img, cv2.COLOR_BGR2GRAY
)

cv2.imshow('Input image', grayScale)
cv2.waitKey(0)
cv2.destroyAllWindows()

Розмиття зображення;

In [4]:
import cv2

img = cv2.imread('./road2.jpg')

gray_scale = cv2.cvtColor(
    img, cv2.COLOR_BGR2GRAY
)

kernel_size = 5
blured_img = cv2.GaussianBlur(
    gray_scale,
    (kernel_size, kernel_size),
    0
)

cv2.imshow('Input image', blured_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Застосування алгоритму Кенні;

In [2]:
import cv2

img = cv2.imread('./road2.jpg')

gray_scale = cv2.cvtColor(
    img, cv2.COLOR_BGR2GRAY
)

kernel_size = 5
blured_img = cv2.GaussianBlur(
    gray_scale,
    (kernel_size, kernel_size),
    0
)

low_t = 50
high_t = 150
edges = cv2.Canny(blured_img, low_t, high_t)

cv2.imshow('Input image', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()


Створення набору вершин для маски;

In [10]:
import cv2
import numpy as np

img = cv2.imread('./road2.jpg')

gray_scale = cv2.cvtColor(
    img, cv2.COLOR_BGR2GRAY
)

kernel_size = 5
blured_img = cv2.GaussianBlur(
    gray_scale,
    (kernel_size, kernel_size),
    0
)

low_t = 50
high_t = 150
edges = cv2.Canny(blured_img, low_t, high_t)

vertices = np.array(
    [
        [(0, img.shape[0]), (450, 310),
         (490, 310), (img.shape[1],
                      img.shape[0])
         ]
    ], dtype=np.int32
)
mask = np.zeros_like(edges)
ignore_mask_color = 255
cv2.fillPoly(mask, vertices, ignore_mask_color)
mask_edges = cv2.bitwise_and(edges, mask)

cv2.imshow('Input image', mask_edges)
cv2.waitKey(0)
cv2.destroyAllWindows()


Маска;

Замасковане зображення;

Застосування перетворення Хафа

In [11]:
import cv2
import numpy as np

img = cv2.imread('./road2.jpg')

def draw_lines(img, lines, color = [255, 0, 0], thickness = 7):
    x_bottom_pos = []
    x_upper_pos = []
    x_bottom_neg = []
    x_upper_neg = []

    y_bottom = 540
    y_upper = 315

    for line in lines:
        for x1, y1, x2, y2 in line:
            if ((y2 - y1) / (x2 - x1)) > 0.5 and ((y2 - y1) / (x2 - x1)) < 0.8:

                slope = ((y2 - y1) / (x2 - x1))
                b = y1 - slope * x1

                x_bottom_pos.append((y_bottom - b) / slope)
                x_upper_pos.append((y_upper - b) / slope)

            elif ((y2 - y1) / (x2 - x1)) < -0.5 and ((y2 - y1) / (x2 - x1)) > -0.8:
                slope = ((y2 - y1) / (x2 - x1))
                b = y1 - slope * x1

                x_bottom_neg.append((y_bottom - b) / slope)
                x_upper_neg.append((y_upper - b) / slope)

    lines_mean = np.array(
        [
            [int(np.mean(x_bottom_pos)), int(np.mean(y_bottom)), int(np.mean(x_upper_pos)), int(np.mean(y_upper))],
            [int(np.mean(x_bottom_neg)), int(np.mean(y_bottom)), int(np.mean(x_upper_neg)), int(np.mean(y_upper))]
        ]
    )

    for i in range(len(lines_mean)):
        cv2.line(img, (lines_mean[i, 0], lines_mean[i, 1]), (lines_mean[i, 2], lines_mean[i, 3]), color, thickness)

low_t = 50
high_t = 255
edges = cv2.Canny(img, low_t, high_t)

vertices = np.array(
    [
        [(0, img.shape[0]), (450, 310),
         (490, 310), (img.shape[1],
                      img.shape[0])
         ]
    ], dtype=np.int32
)
mask = np.zeros_like(edges)
ignore_mask_color = 255
cv2.fillPoly(mask, vertices, ignore_mask_color)
mask_edges = cv2.bitwise_and(edges, mask)

rho = 3
theta = np.pi / 180
threshold = 15
mun_line_len = 150
max_line_gap = 60
lines = cv2.HoughLinesP(
    mask_edges, 
    rho, 
    theta, 
    threshold, 
    np.array([]),
    minLineLength=mun_line_len,
    maxLineGap=max_line_gap
)

draw_lines(img, lines)

cv2.imshow('Input image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Обробка відефайлу

In [9]:
import cv2
import numpy as np

video_capture = cv2.VideoCapture('road.mp4')

def draw_lines(img, lines, color = [255, 0, 0], thickness = 7):
    if lines is None:
        return
    x_bottom_pos, x_upper_pos = [], []
    x_bottom_neg, x_upper_neg = [], []

    y_bottom = img.shape[0]
    y_upper = img.shape[0] * (3/5)

    for line in lines:
        for x1, y1, x2, y2 in line:
            if ((y2 - y1) / (x2 - x1)) > 0.5 and ((y2 - y1) / (x2 - x1)) < 0.8:

                slope = ((y2 - y1) / (x2 - x1))
                b = y1 - slope * x1

                x_bottom_pos.append((y_bottom - b) / slope)
                x_upper_pos.append((y_upper - b) / slope)

            elif ((y2 - y1) / (x2 - x1)) < -0.5 and ((y2 - y1) / (x2 - x1)) > -0.8:
                slope = ((y2 - y1) / (x2 - x1))
                b = y1 - slope * x1

                x_bottom_neg.append((y_bottom - b) / slope)
                x_upper_neg.append((y_upper - b) / slope)

    lines_mean = np.zeros((2, 4), dtype=np.int32)
    if len(x_bottom_pos) > 0 and len(x_upper_pos) > 0:
        lines_mean[0] = [int(np.mean(x_bottom_pos)), int(np.mean(y_bottom)), int(np.mean(x_upper_pos)), int(np.mean(y_upper))]
    if len(x_bottom_neg) > 0 and len(x_upper_neg) > 0:
        lines_mean[1] = [int(np.mean(x_bottom_neg)), int(np.mean(y_bottom)), int(np.mean(x_upper_neg)), int(np.mean(y_upper))]

    for i in range(len(lines_mean)):
        cv2.line(img, (lines_mean[i, 0], lines_mean[i, 1]), (lines_mean[i, 2], lines_mean[i, 3]), color, thickness)

def process_image(frame):
    low_t = 50
    high_t = 255
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    kernel_size = 5
    blur = cv2.GaussianBlur(
        gray,
        (kernel_size, kernel_size),
        0
    )

    edges = cv2.Canny(blur, low_t, high_t)

    vertices = np.array(
        [
            [(0, frame.shape[0]), 
             (450, 310),
             (490, 310), 
             (frame.shape[1], frame.shape[0])
            ]
        ], dtype=np.int32
    )
    mask = np.zeros_like(edges)
    ignore_mask_color = 255
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    mask_edges = cv2.bitwise_and(edges, mask)

    rho = 3
    theta = np.pi / 180
    threshold = 15
    min_line_len = 150
    max_line_gap = 60
    lines = cv2.HoughLinesP(
        mask_edges, 
        rho, 
        theta, 
        threshold, 
        np.array([]),
        minLineLength=min_line_len,
        maxLineGap=max_line_gap
    )

    draw_lines(frame, lines)


while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret: 
        break
    
    process_image(frame)
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

video_capture.release()
cv2.destroyAllWindows()